In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.stats import describe, norm
from math import sqrt

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

def median(lst):
    return sorted(lst)[int(len(lst)/2)]

def conf_int(lst):
    lower = (len(lst)/100)*5
    upper = (len(lst)/100)*95
    lst = sorted(lst)
    return (lst[int(lower)], lst[int(upper)])

def plot_cis(cis):
    mn = 10000
    mx = 0
    for i, ci in enumerate(cis):
        l, u = ci
        mn = min(mn, l-1)
        mx = max(mx, u+1)
    d = mx-mn
    for i, ci in enumerate(cis):
        l, u = ci
        plt.scatter([l, u], [i, i], color=colors[i], lw=2)
        plt.annotate("%.01f"%l, (l, i+0.1), ha="center", va="center")
        plt.annotate("%.01f"%u, (u, i+0.1), ha="center", va="center")
        plt.axhline(y=i, xmin=(l-mn)/d, xmax=(u-mn)/d, color=colors[i])
    plt.xlim(mn, mx)
    plt.ylim(-1, len(cis)+1)
    plt.show()

### Bootstrapped Median Difference

In [ ]:
cs = np.random.normal(loc=130, scale=10, size=10)
non = np.random.normal(loc=130, scale=10, size=10)

plt.hist([cs, non], density=True, label=["cs", "non"])
plt.axvline(median(cs), color=colors[0], ls='--')
plt.axvline(median(non), color=colors[1], ls='--')
plt.legend()
plt.show()


In [ ]:
cs_boots = []
non_boots = []
diff_boots = []
n_iter = 10000
for _ in range(n_iter):
    cs_med = median(np.random.choice(cs, 10))
    cs_boots.append(cs_med)
    non_med = median(np.random.choice(non, 10))
    non_boots.append(non_med)
    diff_boots.append(cs_med - non_med)
    
plt.hist([cs_boots, non_boots], density=True, label=["cs", "non"])
plt.legend()
plt.show()

# compute 95% confidence interval
print(conf_int(cs_boots))
print(conf_int(non_boots))
plot_cis([conf_int(cs_boots), conf_int(non_boots)])

plt.hist(diff_boots, density=True)
plt.show()
print(conf_int(diff_boots))

### Simulation (GMM)

In [ ]:
grades = [60, 55, 65, 66, 70, 58, 60, 68, 54, 88, 74, 76, 80, 55, 60, 90, 91, 89, 93, 95, 88, 85, 96, 89, 95]

# model 1: TAs assign grades randomly centered around 75%
def sim_1(num_iter = 1000):
    lst = np.random.normal(loc=75, scale=5, size=num_iter)
    return lst
    
# model 2: 50/50 split between nice TAs and means ones. Nice ones give grades centered at 90,
# mean ones give grades centered at 60 
def sim_2(num_iter = 1000):
    lst = []
    for _ in range(num_iter):
        coin = np.random.uniform(0,1)
        if coin < 0.5: 
            samp = np.random.normal(loc=60, scale=5, size=1)
            lst.append(samp[0])
        else:
            samp = np.random.normal(loc=90, scale=5, size=1)
            lst.append(samp[0])
    return lst    
    
base = sim_1()
hyp = sim_2()
plt.hist([grades, base, hyp], density=True, label=["observed", "sim1", "sim2"])
plt.legend()
plt.show()

plt.hist(grades, density=True)
plt.show()

In [ ]:
# compute likelihood of observed data under each model
# Note: this is informal, i.e. not a "real" likelihood. Its just meant to give an intuition for how this would work.

# model 1: TAs assign grades randomly centered around 75%
def likelihood_model_1(obs):
    return sum(norm.pdf(obs, loc=75, scale=5))
    
# model 2: 50/50 split between nice TAs and means ones. Nice ones give grades centered at 90,
# mean ones give grades centered at 60 
def likelihood_model_2(obs):
    mean_ta = norm.pdf(obs, loc=60, scale=5)
    nice_ta = norm.pdf(obs, loc=90, scale=5)
    return sum((0.5*mean_ta) + (0.5*nice_ta))

print(likelihood_model_1(grades))
print(likelihood_model_2(grades))